### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

Before running the project, make sure you have the following:

- Install all required dependencies:
  ```bash
  pip install -r requirements.txt

- Create a .env file in the root of the project and add your Groq API key:
```bash
  GROQ_API_KEY=your_api_key_here






In [2]:
## Open AI API key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")

In [8]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="meta-llama/llama-4-scout-17b-16e-instruct",groq_api_key=groq_api_key)
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001FFC50AF440>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001FFC54F44D0>, model_name='meta-llama/llama-4-scout-17b-16e-instruct', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to Spanish"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

AIMessage(content='The translation of "Hello How are you?" from English to Spanish is:\n\n"Hola ¿Cómo estás?" \n\n(or, more formally) \n\n"Hola ¿Cómo está usted?"', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 27, 'total_tokens': 63, 'completion_time': 0.081663083, 'completion_tokens_details': None, 'prompt_time': 0.000146338, 'prompt_tokens_details': None, 'queue_time': 0.043453977, 'total_time': 0.081809421}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_ba95244fa2', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b89b4-c18a-70b2-b7d3-7dcd0912f288-0', usage_metadata={'input_tokens': 27, 'output_tokens': 36, 'total_tokens': 63})

In [22]:
result

AIMessage(content='The translation of "Hello How are you?" from English to Spanish is:\n\n"Hola ¿Cómo estás?" \n\n(or, more formally) \n\n"Hola ¿Cómo está usted?"', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 27, 'total_tokens': 63, 'completion_time': 0.081663083, 'completion_tokens_details': None, 'prompt_time': 0.000146338, 'prompt_tokens_details': None, 'queue_time': 0.043453977, 'total_time': 0.081809421}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_ba95244fa2', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b89b4-c18a-70b2-b7d3-7dcd0912f288-0', usage_metadata={'input_tokens': 27, 'output_tokens': 36, 'total_tokens': 63})

In [23]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'The translation of "Hello How are you?" from English to Spanish is:\n\n"Hola ¿Cómo estás?" \n\n(or, more formally) \n\n"Hola ¿Cómo está usted?"'

In [24]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'The translation of "Hello How are you?" from English to Spanish is:\n\n"Hola ¿Cómo estás?" \n\n(or, more formally)\n"Hola ¿Cómo está usted?"'

In [25]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Translate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",generic_template),
        ("user","{text}")
    ]
)

In [27]:
result=prompt.invoke({"language":"Spanish","text":"Hello"})

In [28]:
result.to_messages()

[SystemMessage(content='Translate the following into Spanish:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [ ]:
## Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"Spanish","text":"Hello"})

'Hola'